# Predição dos resultados da Copa do Mundo 2022

De 4 em 4 anos temos uma competição esportiva mundial que move pessoas de todos os continentes para a frente da televisão com o objetivo de torcer para os seus times preferidos. Algumas pessoas já fazem as suas apostas em quem levará o prêmio para casa, mas com base em que elas tomam essa decisão? Será no histórico de prêmios do time? Nos resultados dos jogos anteriores? No desempenho dos jogadores ao longo dos anos?

Em meio a esse contexto, a pergunta que iremos responder hoje é a seguinte: **Seria possível prever o resultado da Copa do Mundo de 2022?**

> Neste notebook iremos juntar as fontes de dados em uma só para facilitar a análise e construção posterior do modelo de predição do resultado da Copa do Mundo de 2022

Os datasets utilizados nesta análise foram catalogados na Dadosfera e podem ser acessados nos links abaixo:
- [Resultados de jogos internacionais da FIFA](https://app.dadosfera.ai/pt-BR/catalog/data-assets/dd88cab6-b137-45b1-b141-0e93e2235f73)
- [Rankings da FIFA de 1992 a 2022](https://app.dadosfera.ai/pt-BR/catalog/data-assets/49e84cc4-569c-4d73-8b27-4d57dcb640a1)

Referências para a análise: [Kaggle](https://www.kaggle.com/code/sslp23/predicting-fifa-2022-world-cup-with-ml/notebook#Conclusion)

In [1]:
# Importando bibliotecas necessárias
import snowflake.connector
import pandas as pd
import numpy as np
import orchest
import os
from dadosfera import *

### 1. Recuperar os dados do catálogo por meio da biblioteca da Dadosfera

In [2]:
# Recuperando tabelas do catálogo por meio de 'queries'
results = Dadosfera().get_data_from_catalog('TB__DXM7YK__RESULTS')
ranking = Dadosfera().get_data_from_catalog('TB__XX7319__FIFA_RANKING_2022_10_06')

In [ ]:
print(os.getenv('snowflake_password'))

In [6]:
results.head()

,__SDC_ROW,__SDC_SHEET_ID,__SDC_SPREADSHEET_ID,AWAY_SCORE,AWAY_TEAM,CITY,COUNTRY,DATE,HOME_SCORE,HOME_TEAM,NEUTRAL,TOURNAMENT
0,2,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,0.0,England,Glasgow,Scotland,1872-11-30,0.0,Scotland,False,Friendly
1,3,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,2.0,Scotland,London,England,1873-03-08,4.0,England,False,Friendly
2,4,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,1.0,England,Glasgow,Scotland,1874-03-07,2.0,Scotland,False,Friendly
3,5,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,2.0,Scotland,London,England,1875-03-06,2.0,England,False,Friendly
4,6,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,0.0,England,Glasgow,Scotland,1876-03-04,3.0,Scotland,False,Friendly


In [4]:
ranking.head()

,__SDC_ROW,__SDC_SHEET_ID,__SDC_SPREADSHEET_ID,CONFEDERATION,COUNTRY_ABRV,COUNTRY_FULL,PREVIOUS_POINTS,RANK,RANK_CHANGE,RANK_DATE,TOTAL_POINTS
0,2,386821785,15bVMLs2MzTRTaflVu0fPa2R6l8kZLyN_b1MRYWO4XwA,UEFA,GER,Germany,0.0,1.0,0.0,1992-12-31,57.0
1,3,386821785,15bVMLs2MzTRTaflVu0fPa2R6l8kZLyN_b1MRYWO4XwA,AFC,SYR,Syria,0.0,96.0,0.0,1992-12-31,11.0
2,4,386821785,15bVMLs2MzTRTaflVu0fPa2R6l8kZLyN_b1MRYWO4XwA,CAF,BFA,Burkina Faso,0.0,97.0,0.0,1992-12-31,11.0
3,5,386821785,15bVMLs2MzTRTaflVu0fPa2R6l8kZLyN_b1MRYWO4XwA,UEFA,LVA,Latvia,0.0,99.0,0.0,1992-12-31,10.0
4,6,386821785,15bVMLs2MzTRTaflVu0fPa2R6l8kZLyN_b1MRYWO4XwA,CAF,BDI,Burundi,0.0,100.0,0.0,1992-12-31,10.0


### 2. Fazer adaptações nos dois datasets para facilitar a junção dos dois posteriormente

In [5]:
# Renomeando as colunas
results.rename(columns={'AWAY_SCORE': 'away_score', 'AWAY_TEAM':'away_team', 'CITY':'city', 'COUNTRY':'country', 'DATE':'date', 'HOME_SCORE':'home_score', 'HOME_TEAM':'home_team', 'NEUTRAL':'neutral', 'TOURNAMENT':'tournament'}, inplace=True)
ranking.rename(columns={'CONFEDERATION': 'confederation', 'COUNTRY_ABRV':'country_abrv', 'COUNTRY_FULL':'country_full', 'PREVIOUS_POINTS':'previous_points', 'RANK':'rank', 'RANK_CHANGE':'rank_change', 'RANK_DATE':'rank_date', 'TOTAL_POINTS':'total_points'}, inplace=True)
ranking["country_full"] = ranking["country_full"].str.replace("IR Iran", "Iran").str.replace("Korea Republic", "South Korea").str.replace("USA", "United States")

# Adaptando o tipo de dado
results["date"] = pd.to_datetime(results["date"])
ranking["rank_date"] = pd.to_datetime(ranking["rank_date"])

# Resetando indexes
results = results.reset_index(drop=True)
ranking = ranking.reset_index(drop=True)

In [6]:
results.head()

,__SDC_ROW,__SDC_SHEET_ID,__SDC_SPREADSHEET_ID,away_score,away_team,city,country,date,home_score,home_team,neutral,tournament
0,2,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,0.0,England,Glasgow,Scotland,1872-11-30,0.0,Scotland,False,Friendly
1,3,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,2.0,Scotland,London,England,1873-03-08,4.0,England,False,Friendly
2,4,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,1.0,England,Glasgow,Scotland,1874-03-07,2.0,Scotland,False,Friendly
3,5,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,2.0,Scotland,London,England,1875-03-06,2.0,England,False,Friendly
4,6,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,0.0,England,Glasgow,Scotland,1876-03-04,3.0,Scotland,False,Friendly


### 3. Juntar os datasets em um só
Cada linha da tabela de Resultados contém o resultado de um jogo e cada linha da tabela de Ranking possui um país e a sua respectiva posição e pontuação no ranking. O que faremos agora é juntar os dois datasets, de forma que cada linha contenha tanto os dados de resultado de um jogo quando dados de ranking dos dois países adversários no momento da partida.

In [7]:
# Separando dados por data do ranking
ranking = ranking.set_index(['rank_date']
                           ).groupby(['country_full'], group_keys=False).resample('D').first().fillna(method='ffill').reset_index()

# Adicionando dados do time da casa
df = results.merge(ranking[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], 
                   left_on=["date", "home_team"], 
                   right_on=["rank_date", "country_full"]
                  ).drop(["rank_date", "country_full"], axis=1)

# Adicionando dados do time que não é da casa
df = df.merge(ranking[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], 
              left_on=["date", "away_team"], 
              right_on=["rank_date", "country_full"], 
              suffixes=("_home", "_away")
             ).drop(["rank_date", "country_full"], axis=1)

df.head()

,__SDC_ROW,__SDC_SHEET_ID,__SDC_SPREADSHEET_ID,away_score,away_team,city,country,date,home_score,home_team,neutral,tournament,total_points_home,previous_points_home,rank_home,rank_change_home,total_points_away,previous_points_away,rank_away,rank_change_away
0,17699,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,1.0,Mali,Libreville,Gabon,1993-01-01,1.0,Ghana,True,Friendly,34.0,0.0,39.0,0.0,22.0,0.0,69.0,0.0
1,17700,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,1.0,Burkina Faso,Libreville,Gabon,1993-01-02,1.0,Gabon,False,Friendly,27.0,0.0,55.0,0.0,11.0,0.0,97.0,0.0
2,17702,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,0.0,Mali,Libreville,Gabon,1993-01-03,1.0,Burkina Faso,True,Friendly,11.0,0.0,97.0,0.0,22.0,0.0,69.0,0.0
3,17703,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,3.0,Ghana,Libreville,Gabon,1993-01-03,2.0,Gabon,False,Friendly,27.0,0.0,55.0,0.0,34.0,0.0,39.0,0.0
4,17704,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,1.0,Tanzania,Kampala,Uganda,1993-01-08,3.0,Uganda,False,Friendly,12.0,0.0,92.0,0.0,15.0,0.0,80.0,0.0


In [8]:
print(f'Dimensões do dataset de resultados: {results.shape}')
print(f'Dimensões do dataset de ranking: {ranking.shape}\n')

print(f'Dimensões do novo dataset: {df.shape}')

Dimensões do dataset de resultados: (44206, 12)
Dimensões do dataset de ranking: (2186619, 11)

Dimensões do novo dataset: (22684, 20)


### 4. Exportar o dataset criado para o próximo passo

In [9]:
dadosfera_utils.output(df, name="df")